# Aula 16 - Autoencoders

## Fundamentos e Aplicações de Autoencoders

### O que são Autoencoders?
- **Definição:** 
  - Redes neurais usadas para aprender representações de dados, geralmente para redução de dimensionalidade. Elas tentam capturar as características mais importantes dos dados de entrada, de modo a poder reconstruí-los com boa precisão.
- **Arquitetura:** 
  - Composta por duas partes principais: codificador (encoder) e decodificador (decoder). O codificador transforma os dados de entrada em uma representação de menor dimensão (codificação latente), e o decodificador reconstrói os dados de entrada a partir dessa codificação.

### Componentes de um Autoencoder
- **Codificador (Encoder):**
  - Reduz a dimensão dos dados de entrada.
  - **Função de ativação comum:** ReLU (Rectified Linear Unit), que introduz não-linearidade ao modelo, permitindo que ele aprenda representações mais complexas.
  - **Estrutura típica:** Composta por camadas densas (fully connected layers), onde cada camada subsequente possui menos neurônios do que a anterior, forçando a rede a aprender uma representação comprimida dos dados.

- **Codificação Latente:**
  - Representação compacta e comprimida dos dados de entrada.
  - **Tamanho da camada latente:** Determinado pela necessidade específica da aplicação e pela quantidade de informação que deve ser retida. Deve ser um compromisso entre a capacidade de compressão e a precisão de reconstrução.

- **Decodificador (Decoder):**
  - Reconstrói os dados originais a partir da codificação latente.
  - **Função de ativação comum:** Sigmoid ou Tanh, que são frequentemente utilizadas nas camadas finais para garantir que os valores de saída estejam no mesmo intervalo que os dados de entrada.
  - **Estrutura típica:** Semelhante ao codificador, mas em ordem inversa. As camadas possuem um número crescente de neurônios, expandindo a codificação latente de volta ao formato original dos dados de entrada.

### Processo de Treinamento
- **Função de Custo:** 
  - O objetivo do treinamento é minimizar o erro de reconstrução, que é a diferença entre os dados de entrada e os dados reconstruídos pelo autoencoder. Uma função de custo comum é o erro quadrático médio (MSE - Mean Squared Error).
  - **Erro de reconstrução:** Mede o quanto os dados de saída se desviam dos dados de entrada.

- **Algoritmo de Otimização:**
  - Uso do algoritmo de descida do gradiente (Gradient Descent) para ajustar os pesos da rede neural de forma a minimizar a função de custo.
  - **Learning rate:** Taxa de aprendizado que controla o tamanho dos passos dados na direção do gradiente negativo. Um learning rate adequado é crucial para garantir a convergência eficiente e estável do modelo.

### Aplicações de Autoencoders
- **Redução de Dimensionalidade:** 
  - Extração de características principais dos dados de entrada, facilitando a visualização, compressão e análise dos dados.
  - Compressão de dados, permitindo armazenar grandes volumes de dados em um formato mais compacto.

- **Detecção de Anomalias:**
  - Identificação de dados que não se ajustam ao padrão normal, com base no erro de reconstrução. Dados que resultam em alto erro de reconstrução podem ser considerados anômalos.

- **Pré-treinamento para Redes Neurais Profundas:**
  - Inicialização de pesos para outras redes neurais, melhorando a eficiência e a eficácia do treinamento em tarefas subsequentes.


## Redução de Dimensionalidade de uma Tabela com Autoencoders

### Passos para Reduzir a Dimensionalidade de uma Tabela com Autoencoders

#### 1. Pré-processamento dos Dados
- **Normalização:** Escale seus dados para que todas as características estejam na mesma faixa de valores (por exemplo, entre 0 e 1).
  - Isso ajuda o modelo a convergir mais rapidamente e a melhorar a precisão da reconstrução.

#### 2. Definição do Modelo Autoencoder
- **Codificador (Encoder):** 
  - Reduz a dimensão dos dados de entrada.
  - **Exemplo:** Se sua tabela original tem 100 características, você pode querer reduzir para 10 características principais.
- **Camada Latente:**
  - A camada de codificação compacta os dados para um espaço de dimensão menor.
  - **Exemplo:** Um vetor de 10 valores.
- **Decodificador (Decoder):**
  - Reconstrói os dados originais a partir da codificação latente.
  - A estrutura do decodificador deve ser simétrica ao codificador.

#### 3. Treinamento do Autoencoder
- **Função de Custo:** 
  - Utilize uma função de custo como o erro quadrático médio (MSE) para medir a diferença entre os dados de entrada e os dados reconstruídos.
- **Algoritmo de Otimização:**
  - Utilize o algoritmo de descida do gradiente com uma taxa de aprendizado adequada.
- **Treinamento:**
  - Treine o autoencoder com seus dados até que a função de custo seja minimizada e a reconstrução seja suficientemente precisa.

#### 4. Extração das Características Latentes
- **Utilize apenas o Codificador:**
  - Após o treinamento, utilize apenas a parte do codificador do autoencoder para transformar seus dados de entrada na representação de baixa dimensionalidade.
- **Novo Conjunto de Dados:**
  - O resultado será uma nova tabela com o número de características reduzido, mas preservando a maior parte da informação essencial dos dados originais.


In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

2024-06-04 08:33:58.310054: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
df = pd.read_csv('weather_data.csv')

In [3]:
df.head()

,Location,Date_Time,Temperature_C,Humidity_pct,Precipitation_mm,Wind_Speed_kmh
0,San Diego,2024-01-14 21:12:46,10.683001,41.195754,4.020119,8.233540
1,San Diego,2024-05-17 15:22:10,8.734140,58.319107,9.111623,27.715161
2,San Diego,2024-05-11 09:30:59,11.632436,38.820175,4.607511,28.732951
3,Philadelphia,2024-02-26 17:32:39,-8.628976,54.074474,3.183720,26.367303
4,San Antonio,2024-04-29 13:23:51,39.808213,72.899908,9.598282,29.898622


In [4]:
# Selecionar apenas as colunas numéricas para a normalização e o autoencoder
numeric_columns = ['Temperature_C', 'Humidity_pct', 'Precipitation_mm', 'Wind_Speed_kmh']
data = df[numeric_columns]

In [5]:
# Normalização dos dados
scaler = MinMaxScaler()
data_normalized = scaler.fit_transform(data)

In [6]:
# Definição do Autoencoder
input_dim = data_normalized.shape[1]
encoding_dim = 2  # Reduzir para 2 características principais

In [7]:
# Definindo a entrada do autoencoder
# input_layer define a camada de entrada com o formato dos dados de entrada (input_dim características)
input_layer = Input(shape=(input_dim,))

# Definindo a camada de codificação (encoder)
# encoder é uma camada densa com encoding_dim neurônios e ativação ReLU
# Esta camada reduz a dimensionalidade dos dados de entrada
encoder = Dense(encoding_dim, activation='relu')(input_layer)

# Definindo a camada de decodificação (decoder)
# decoder é uma camada densa com o mesmo número de neurônios que a camada de entrada e ativação sigmoid
# Esta camada reconstrói os dados para o formato original
decoder = Dense(input_dim, activation='sigmoid')(encoder)

# Construindo o modelo autoencoder
# autoencoder é o modelo que mapeia os dados de entrada para a saída reconstruída
autoencoder = Model(inputs=input_layer, outputs=decoder)

# Compilação do Autoencoder
# O compilador configura o modelo para o treinamento
# optimizer: algoritmo de otimização (Adam é um método de descida de gradiente estocástico)
# loss: função de perda (mean_squared_error calcula o erro quadrático médio entre a entrada e a saída reconstruída)
autoencoder.compile(optimizer='adam', loss='mean_squared_error')


In [8]:
# Treinamento do Autoencoder
autoencoder.fit(data_normalized, data_normalized, epochs=50, batch_size=256, shuffle=True)


Epoch 1/50
3907/3907 ━━━━━━━━━━━━━━━━━━━━ 5s 872us/step - loss: 0.0485
Epoch 2/50
3907/3907 ━━━━━━━━━━━━━━━━━━━━ 3s 838us/step - loss: 0.0248
Epoch 3/50
3907/3907 ━━━━━━━━━━━━━━━━━━━━ 4s 953us/step - loss: 0.0247
Epoch 4/50
3907/3907 ━━━━━━━━━━━━━━━━━━━━ 3s 815us/step - loss: 0.0247
Epoch 5/50
3907/3907 ━━━━━━━━━━━━━━━━━━━━ 3s 797us/step - loss: 0.0247
Epoch 6/50
3907/3907 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - loss: 0.0247
Epoch 7/50
3907/3907 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - loss: 0.0247
Epoch 8/50
3907/3907 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - loss: 0.0247
Epoch 9/50
3907/3907 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - loss: 0.0247
Epoch 10/50
3907/3907 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - loss: 0.0247
Epoch 11/50
3907/3907 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - loss: 0.0247
Epoch 12/50
3907/3907 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - loss: 0.0247
Epoch 13/50
3907/3907 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - loss: 0.0247
Epoch 14/50
3907/3907 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - loss: 0.0247
Epoch 15/50
3907/39

In [9]:
# Extração da camada de codificação
encoder_model = Model(inputs=input_layer, outputs=encoder)
data_reduced = encoder_model.predict(data_normalized)

31250/31250 ━━━━━━━━━━━━━━━━━━━━ 27s 861us/step


In [10]:
# Transformar a tabela reduzida em um DataFrame
df_reduced = pd.DataFrame(data_reduced, columns=['Feature1', 'Feature2'])

In [11]:
# Exibir as primeiras linhas da tabela reduzida
df_reduced.head()

,Feature1,Feature2
0,0.697906,0.560127
1,0.934213,1.751381
2,0.288642,1.404904
3,0.825730,1.616735
4,1.357215,2.120519


In [12]:
# Para reconstruir as colunas originais a partir dos dados reduzidos:
# Definir o modelo de decodificação
encoded_input = Input(shape=(encoding_dim,))
decoder_layer = autoencoder.layers[-1]  # Última camada do autoencoder
decoder_model = Model(inputs=encoded_input, outputs=decoder_layer(encoded_input))

In [13]:
# Reconstruir os dados originais
data_reconstructed = decoder_model.predict(data_reduced)

# Reverter a normalização
data_reconstructed = scaler.inverse_transform(data_reconstructed)

# Transformar os dados reconstruídos em um DataFrame
df_reconstructed = pd.DataFrame(data_reconstructed, columns=numeric_columns)

# Exibir as primeiras linhas da tabela reconstruída
df_reconstructed.head()

31250/31250 ━━━━━━━━━━━━━━━━━━━━ 27s 846us/step


,Temperature_C,Humidity_pct,Precipitation_mm,Wind_Speed_kmh
0,15.104754,40.567528,5.107385,7.435331
1,14.582826,57.947834,5.135482,26.707541
2,14.776352,38.975212,5.125752,27.152895
3,14.647844,53.149559,5.132082,26.049852
4,14.395300,74.114098,5.145144,27.556599


In [14]:
# Exibe os dados originais
data.head()

,Temperature_C,Humidity_pct,Precipitation_mm,Wind_Speed_kmh
0,10.683001,41.195754,4.020119,8.233540
1,8.734140,58.319107,9.111623,27.715161
2,11.632436,38.820175,4.607511,28.732951
3,-8.628976,54.074474,3.183720,26.367303
4,39.808213,72.899908,9.598282,29.898622


OBS: Assim como em outras redes neurais, aqui também lidamos apenas com os dados numéricos 

In [15]:
# Função para calcular o tamanho total em memória de um DataFrame
def memory_usage_df(df):
    return df.memory_usage(deep=True).sum() / (1024 * 1024)

In [16]:
# Tamanho em memória das variáveis
size_data = memory_usage_df(data)
size_df_reduced = memory_usage_df(df_reduced)
size_df_reconstructed = memory_usage_df(df_reconstructed)


print(f'Tamanho de "data" em memória: {size_data:.2f} MB')
print(f'Tamanho de "df_reduced" em memória: {size_df_reduced:.2f} MB')
print(f'Tamanho de "df_reconstructed" em memória: {size_df_reconstructed:.2f} MB')

Tamanho de "data" em memória: 30.52 MB
Tamanho de "df_reduced" em memória: 7.63 MB
Tamanho de "df_reconstructed" em memória: 15.26 MB


# Comparação de DataFrames Usando Métricas de Reconstrução

## Função `compare_dataframes`

Esta função compara dois DataFrames, um original e um reconstruído, e calcula as métricas de qualidade de reconstrução para cada coluna. As métricas calculadas são:
- **Erro Quadrático Médio (MSE)**
- **Erro Absoluto Médio (MAE)**
- **Coeficiente de Determinação (R² Score)**

### Parâmetros
- `original_df` (DataFrame): O DataFrame original com os dados antes da redução de dimensionalidade.
- `reconstructed_df` (DataFrame): O DataFrame reconstruído a partir da representação de baixa dimensionalidade.

### Retorno
- DataFrame com as colunas:
  - `Column`: Nome da coluna.
  - `MSE`: Erro Quadrático Médio para a coluna.
  - `MAE`: Erro Absoluto Médio para a coluna.
  - `R²`: Coeficiente de Determinação para a coluna.

In [17]:
def compare_dataframes(original_df, reconstructed_df):
    results = []
    
    for column in original_df.columns:
        mse = mean_squared_error(original_df[column], reconstructed_df[column])
        mae = mean_absolute_error(original_df[column], reconstructed_df[column])
        r2 = r2_score(original_df[column], reconstructed_df[column])
        
        results.append({
            'Column': column,
            'MSE': mse,
            'MAE': mae,
            'R²': r2
        })
    
    return pd.DataFrame(results)

In [18]:
# Aqui você deve ter os dataframes 'data' e 'df_reconstructed'
comparison_results = compare_dataframes(data, df_reconstructed)

# Exibir os resultados
print(comparison_results)


             Column         MSE        MAE        R²
0     Temperature_C  209.682369  12.530101  0.000295
1      Humidity_pct    3.043488   1.420522  0.989859
2  Precipitation_mm    8.690957   2.536440 -0.000032
3    Wind_Speed_kmh    0.755801   0.711392  0.989930


### Utilização de Autoencoders com Diferentes Tipos de Dados

#### Dados Numéricos Contínuos
- **Aplicação:** Autoencoders são altamente eficazes para dados numéricos contínuos, como medidas de temperatura, umidade, preços, etc.
- **Vantagens:** Redução de dimensionalidade, extração de características principais e compressão de dados.

#### Dados Numéricos Inteiros
- **Aplicação:** Autoencoders podem ser usados para dados inteiros que representam quantidades, como idade, contagem de itens, etc.
- **Considerações:** Normalização pode ser necessária para melhorar a performance do autoencoder.

#### Dados Numéricos que Representam Classes
- **Aplicação:** Dados inteiros que representam categorias (como níveis educacionais, códigos de produtos) podem não ser adequados para autoencoders diretamente.
- **Observação:** Nesses casos, técnicas de pré-processamento como one-hot encoding ou embeddings são recomendadas.

#### Dados Textuais
- **Aplicação:** Autoencoders não são diretamente aplicáveis a dados textuais brutos.
- **Observação:** Dados textuais precisam ser convertidos em representações numéricas (como embeddings) antes de serem usados com autoencoders.

### Conclusão
Autoencoders são ferramentas poderosas para a redução de dimensionalidade e extração de características, especialmente com dados numéricos contínuos. Para dados inteiros e categóricos, transformações adicionais podem ser necessárias. Dados textuais requerem etapas de pré-processamento para serem utilizados de forma eficaz com autoencoders.


# Exercícios com base de dados preço de casas na California


In [19]:
df = pd.read_csv('housing.csv')
df.head()

,Unnamed: 0,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   20640 non-null  int64  
 1   MedInc       20640 non-null  float64
 2   HouseAge     20640 non-null  float64
 3   AveRooms     20640 non-null  float64
 4   AveBedrms    20640 non-null  float64
 5   Population   20640 non-null  float64
 6   AveOccup     20640 non-null  float64
 7   Latitude     20640 non-null  float64
 8   Longitude    20640 non-null  float64
 9   MedHouseVal  20640 non-null  float64
dtypes: float64(9), int64(1)
memory usage: 1.6 MB


In [21]:
df.describe()

,Unnamed: 0,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
count,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,10319.500000,3.870671,28.639486,5.429000,1.096675,1425.476744,3.070655,35.631861,-119.569704,2.068558
std,5958.399114,1.899822,12.585558,2.474173,0.473911,1132.462122,10.386050,2.135952,2.003532,1.153956
min,0.000000,0.499900,1.000000,0.846154,0.333333,3.000000,0.692308,32.540000,-124.350000,0.149990
25%,5159.750000,2.563400,18.000000,4.440716,1.006079,787.000000,2.429741,33.930000,-121.800000,1.196000
50%,10319.500000,3.534800,29.000000,5.229129,1.048780,1166.000000,2.818116,34.260000,-118.490000,1.797000
75%,15479.250000,4.743250,37.000000,6.052381,1.099526,1725.000000,3.282261,37.710000,-118.010000,2.647250
max,20639.000000,15.000100,52.000000,141.909091,34.066667,35682.000000,1243.333333,41.950000,-114.310000,5.000010


1 - Exclua as colunas 'Unnamed: 0', 'Latitude' e 'Longitude'.



In [22]:
df.drop(columns=['Unnamed: 0', 'Latitude', 'Longitude'], axis=1, inplace=True)

2 - Normalize a base de dados com MinMaxScaler.



In [23]:
scaler = MinMaxScaler()
data_normalized = scaler.fit_transform(df)

3 - Defina e compile um modelo de autoencoder para reduzir para 3 colunas (encoding_dim = 3).



In [24]:
input_dim = data_normalized.shape[1]
encoding_dim = 3 

4 - Treine o modelo por 50 épocas com batch_size de 64.



In [26]:
# Definindo a entrada do autoencoder
# input_layer define a camada de entrada com o formato dos dados de entrada (input_dim características)
input_layer = Input(shape=(input_dim,))

# Definindo a camada de codificação (encoder)
# encoder é uma camada densa com encoding_dim neurônios e ativação ReLU
# Esta camada reduz a dimensionalidade dos dados de entrada
encoder = Dense(encoding_dim, activation='relu')(input_layer)

# Definindo a camada de decodificação (decoder)
# decoder é uma camada densa com o mesmo número de neurônios que a camada de entrada e ativação sigmoid
# Esta camada reconstrói os dados para o formato original
decoder = Dense(input_dim, activation='sigmoid')(encoder)

# Construindo o modelo autoencoder
# autoencoder é o modelo que mapeia os dados de entrada para a saída reconstruída
autoencoder = Model(inputs=input_layer, outputs=decoder)

# Compilação do Autoencoder
# O compilador configura o modelo para o treinamento
# optimizer: algoritmo de otimização (Adam é um método de descida de gradiente estocástico)
# loss: função de perda (mean_squared_error calcula o erro quadrático médio entre a entrada e a saída reconstruída)
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

In [27]:
autoencoder.fit(data_normalized, data_normalized, epochs=50, batch_size=256, shuffle=True)

Epoch 1/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.1541   
Epoch 2/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1277
Epoch 3/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0892
Epoch 4/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0568
Epoch 5/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0395
Epoch 6/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0310
Epoch 7/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0263
Epoch 8/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0235
Epoch 9/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0215
Epoch 10/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0203
Epoch 11/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0191
Epoch 12/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0180
Epoch 13/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0171
Epoch 14/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0159
Epoch 15/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0149
Epoch 16/50
81/8

5 - Faça a extração da camada de codificação (previsão com o modelo).



In [29]:
encoder_model = Model(inputs=input_layer, outputs=encoder)
data_reduced = encoder_model.predict(data_normalized)

645/645 ━━━━━━━━━━━━━━━━━━━━ 1s 912us/step


6 - Crie um DataFrame reduzido com as 3 colunas.



In [30]:
df_reduced = pd.DataFrame(data_reduced, columns=['Feature1', 'Feature2', 'Feature3'])

7 - Defina o decodificador para converter os dados reduzidos para o formato original.



In [31]:
# Definir o modelo de decodificação
encoded_input = Input(shape=(encoding_dim,))
decoder_layer = autoencoder.layers[-1]  # Última camada do autoencoder
decoder_model = Model(inputs=encoded_input, outputs=decoder_layer(encoded_input))

8 - Reconstrua o DataFrame reduzido para a estrutura do original (usando as mesmas colunas).



In [33]:
# Reconstruir os dados originais
data_reconstructed = decoder_model.predict(data_reduced)

# Reverter a normalização
data_reconstructed = scaler.inverse_transform(data_reconstructed)

# Transformar os dados reconstruídos em um DataFrame
df_reconstructed = pd.DataFrame(data_reconstructed, columns=df.columns)

# Exibir as primeiras linhas da tabela reconstruída
df_reconstructed.head()

645/645 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step


,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,MedHouseVal
0,8.075545,41.557816,2.019270,0.453757,647.549805,3.935999,4.549531
1,7.436614,19.692274,3.208966,0.534403,968.033020,7.545003,4.100762
2,5.918516,47.491276,2.061484,0.533253,689.895386,4.195951,4.073101
3,5.027290,47.602516,2.172661,0.584057,724.516296,4.546252,3.701489
4,4.191662,47.656334,2.193995,0.622497,712.292175,4.548414,3.291351


9 - Exiba o tamanho em memória do DataFrame original, do DataFrame reduzido e do DataFrame reconstruído.

In [34]:
data.head()

,Temperature_C,Humidity_pct,Precipitation_mm,Wind_Speed_kmh
0,10.683001,41.195754,4.020119,8.233540
1,8.734140,58.319107,9.111623,27.715161
2,11.632436,38.820175,4.607511,28.732951
3,-8.628976,54.074474,3.183720,26.367303
4,39.808213,72.899908,9.598282,29.898622


In [39]:
# Tamanho em memória das variáveis
size_data = memory_usage_df(df)
size_df_reduced = memory_usage_df(df_reduced)
size_df_reconstructed = memory_usage_df(df_reconstructed)


print(f'Tamanho de "df" em memória: {size_data:.2f} MB')
print(f'Tamanho de "df_reduced" em memória: {size_df_reduced:.2f} MB')
print(f'Tamanho de "df_reconstructed" em memória: {size_df_reconstructed:.2f}MB')

Tamanho de "df" em memória: 1.10 MB
Tamanho de "df_reduced" em memória: 0.24 MB
Tamanho de "df_reconstructed" em memória: 0.55MB


10 - Exiba as métricas comparando o DataFrame original e o reconstruído.

In [36]:
def compare_dataframes(original_df, reconstructed_df):
    results = []
    
    for column in original_df.columns:
        mse = mean_squared_error(original_df[column], reconstructed_df[column])
        mae = mean_absolute_error(original_df[column], reconstructed_df[column])
        r2 = r2_score(original_df[column], reconstructed_df[column])
        
        results.append({
            'Column': column,
            'MSE': mse,
            'MAE': mae,
            'R²': r2
        })
    
    return pd.DataFrame(results)

In [38]:
# Aqui você deve ter os dataframes 'df' e 'df_reconstructed'
comparison_results = compare_dataframes(df, df_reconstructed)

# Exibir os resultados
print(comparison_results)

        Column           MSE         MAE        R²
0       MedInc  6.313927e-01    0.602092  0.825058
1     HouseAge  2.387869e+00    1.101141  0.984924
2     AveRooms  1.007373e+01    1.930842 -0.645702
3    AveBedrms  3.802351e-01    0.359471 -0.693090
4   Population  1.438965e+06  754.875215 -0.122080
5     AveOccup  2.538544e+02   10.611564 -1.353450
6  MedHouseVal  9.059733e-02    0.232371  0.931961
